Authored by: Aryan Mistry

# Prompt Engineering 101

Prompt engineering is the art and science of crafting instructions that guide a language model toward desirable outputs. Because models rely solely on the text you provide to decide what to do, the prompt becomes your **program** for the model. A well‑crafted prompt specifies the task clearly, provides any relevant context, may include examples, and defines how the output should be structured.

Throughout this notebook you'll learn core techniques for designing prompts effectively and experiment with them through simple code examples. [16]

## System vs. User Prompts

Large language models often distinguish between a *system* message—describing the assistant’s role or behaviour—and a *user* message containing the actual request. The system prompt sets global behaviour (e.g. "You are a helpful assistant who translates English to French"), while the user prompt contains the specific query.

When composing prompts:

- **Be clear about the role:** tell the model who it should be (e.g., tutor, proofreader).
- **Specify the task:** summarise, translate, answer, brainstorm, etc.
- **Provide constraints:** tone, output format, style guidelines.
- **Include examples when needed:** few‑shot prompts can steer the model toward desired behaviour. [16]

In [1]:
def simple_chat(system_prompt: str, user_prompt: str) -> str:
    """A toy chat function that changes its behaviour based on the system prompt.
    This is only illustrative and does not call a real language model.
    """
    system_prompt = system_prompt.lower()
    user_prompt = user_prompt.strip()

    if 'translate' in system_prompt and 'english to french' in system_prompt:
        # naive word-by-word translation dictionary
        dictionary = {'hello': 'bonjour', 'world': 'monde', 'friend': 'ami'}
        words = user_prompt.lower().split()
        return ' '.join(dictionary.get(w, w) for w in words)
    elif 'polite' in system_prompt:
        return f"I would be pleased to assist you with: {user_prompt}"
    else:
        return f"Echo: {user_prompt}"

# Examples
resp1 = simple_chat('You are a helpful assistant that translates English to French.', 'Hello world')
resp2 = simple_chat('You are a polite assistant.', 'Provide the schedule for tomorrow.')
resp3 = simple_chat('You are an assistant.', 'Tell me a joke.')
print(resp1)
print(resp2)
print(resp3)

bonjour monde
I would be pleased to assist you with: Provide the schedule for tomorrow.
Echo: Tell me a joke.


In the toy `simple_chat` function above the behaviour changes depending on the system prompt. When instructed to translate English to French it looks up individual words. When told to be polite it wraps the user request in a courteous response. Otherwise it simply echoes the request. Although rudimentary, this illustrates how a system message influences the output of a language model.

## Few‑Shot Prompting

Few‑shot prompts provide the model with one or more examples of the desired input–output behaviour. The model infers the pattern and applies it to new inputs. Below we implement a simple few‑shot translator that learns mappings from examples provided in the prompt. The function parses the examples and applies them as a lookup; a real model would generalise beyond exact matches. [16]

In [2]:
def few_shot_translate(examples: list[tuple[str, str]], new_sentence: str) -> str:
    """
    A naive few-shot translator that uses provided examples to map words.
    Examples should be a list of (source, target) tuples.
    """
    mapping = {src.lower(): tgt for src, tgt in examples}
    words = new_sentence.lower().split()
    return ' '.join(mapping.get(w, w) for w in words)

# Provide a few examples
examples = [('hello', 'hola'), ('friend', 'amigo'), ('world', 'mundo')]
print(few_shot_translate(examples, 'hello friend'))
print(few_shot_translate(examples, 'goodbye world'))

hola amigo
goodbye mundo


This few‑shot translator simply looks up each word in the provided examples. It does not infer grammar or unseen vocabulary. Real language models use the examples to condition their hidden representations and can generalise to new phrases.

## Guidelines for Effective Prompting

When designing prompts it helps to think like a teacher: set expectations, provide examples and define the desired output. Some general principles include:

1. **Start simple and iterate:** Begin with a straightforward instruction. Try the prompt, examine the output, and iteratively refine your instructions. Avoid writing a complicated prompt all at once.
2. **Be explicit about the task:** State clearly what you want the model to do (e.g. "Translate the following sentence into French", "Summarize this paragraph in one sentence", "Classify the sentiment as positive, negative or neutral").
3. **Provide relevant context:** Supply background information or constraints so the model does not have to guess, such as assumptions about the audience or domain-specific knowledge.
4. **Specify the output format:** If you need a numbered list, a JSON object, bullet points or a particular tone, say so explicitly.
5. **Use examples when appropriate:** Few‑shot examples (input–output pairs) illustrate the pattern you expect and can improve performance on classification, translation and transformation tasks.
6. **Give positive instructions:** Phrase your request in terms of what the model should do rather than what it should avoid. For example, "Recommend five movies suitable for a family audience" rather than "Don't recommend any movies that are not family friendly".
7. **Avoid ambiguity and cleverness:** Use simple, direct language rather than puns or sarcasm. If a prompt could be misinterpreted, add clarifying details.

The following sections expand on these principles with concrete examples and exercises. [16]

## Prompt Elements: Instruction, Context, Examples, Output Format

A good prompt often includes four key elements:

* **Instruction:** A clear description of what you want the model to do (e.g. "Summarize the following text in one sentence").
* **Context:** Any background information or constraints the model needs to perform the task (e.g. "Assume the reader has a basic knowledge of physics").
* **Examples:** One or more input→output pairs that illustrate the desired behaviour. This is called *few‑shot prompting*.
* **Output format:** Explicit directions on how to structure the response (e.g. "Return the answer as a JSON object with keys `name` and `category`", or "Provide the summary in bullet points").

By composing these elements, you can dramatically improve the quality of the model’s response. The next sections show how to apply these ideas in practice. [16]

## Zero‑Shot and Few‑Shot Prompting

**Zero‑shot prompting** means asking the model to perform a task without providing any examples. You rely entirely on the instruction and context. This works surprisingly well for many tasks, but the model may misinterpret your intent if you aren't specific.

**Few‑shot prompting** embeds a few input→output examples directly in the prompt. The model sees the pattern and generalises it to new inputs. This technique can boost performance on tasks like translation, classification or extraction. When writing few‑shot prompts, make sure your examples are representative of the task and your labels are consistent. [16]

In [3]:
def zero_shot_sentiment(sentence: str) -> str:
    # A naive zero-shot sentiment classifier that uses keyword heuristics.
    positive_words = {'love', 'delicious', 'good', 'happy', 'excellent'}
    negative_words = {'hate', 'bad', 'terrible', 'awful', 'horrible'}
    words = set(sentence.lower().split())
    if words & positive_words and not (words & negative_words):
        return 'positive'
    elif words & negative_words and not (words & positive_words):
        return 'negative'
    else:
        return 'neutral'


def few_shot_sentiment(examples: list[tuple[str, str]], new_sentence: str) -> str:
    # A naive few-shot classifier: uses examples to build a lookup table.
    label_vocab = {}
    for sent, label in examples:
        for word in sent.lower().split():
            label_vocab[word] = label
    votes = {'positive': 0, 'negative': 0, 'neutral': 0}
    for word in new_sentence.lower().split():
        if word in label_vocab:
            votes[label_vocab[word]] += 1
    return max(votes, key=votes.get)


# Compare zero-shot vs few-shot on an example sentence
sentence = "The food was delicious and the service excellent"
print('Zero-shot result:', zero_shot_sentiment(sentence))
print('Few-shot result:', few_shot_sentiment([
    ("I love this product", "positive"),
    ("This is terrible", "negative")
], sentence))

Zero-shot result: positive
Few-shot result: positive


## Examples: Translation, Summarization, and Classification

Below are some example prompts that illustrate how clarity and specificity affect model outputs.

* **Translation:**

  "Translate the following sentence into French:

    'The meeting will take place tomorrow afternoon.'"

  By stating the task ("Translate") and specifying the target language, you reduce ambiguity.

* **Summarization:**

  "In one or two sentences, summarize the main points of the paragraph below, focusing on the major events:

    [insert paragraph here]"

  This prompt instructs the model to produce a concise summary and clarifies what to focus on.

* **Classification:**

  "Classify the sentiment of the sentence below as 'positive', 'negative', or 'neutral':

    'I waited 45 minutes for my food and it was cold when it arrived.'

Answer with just the label."

  The prompt states the task, the possible labels, the input, and the desired output format.

* **Structured Output:**

  "Extract the chemical names from the paragraph below and return them as a JSON array under the key `chemicals`:

    [insert paragraph here]"

  Here you combine instruction, context, and explicit output formatting to guide the model. [16]

## Putting It into Practice

The toy functions defined above (`simple_chat` and `few_shot_translate`) can be used to experiment with prompt structure. For example, you can change the system prompt to instruct the model to reply with exaggerated enthusiasm or to answer only in the style of a pirate. Because these functions are extremely simple, they provide immediate feedback about how prompt changes affect behaviour.

In a real LLM setting you would replace these toy functions with calls to an API or library such as `openai.ChatCompletion` or Hugging Face’s `pipeline`. The same principles still apply.

In [4]:
# Experiment with system and user prompts.
# Try changing the system prompt to different personas and see how the reply changes.
print('Default behaviour:', simple_chat('you are a neutral assistant', 'What is the capital of France?'))
print('Polite persona:', simple_chat('You are a very polite assistant.', 'What is the capital of France?'))

# Now build a few-shot translation example
examples = [('hello', 'hola'), ('world', 'mundo'), ('cat', 'gato'), ('dog', 'perro')]
print('Few-shot translation:', few_shot_translate(examples, 'hello world'))

# TODO: Extend this example by adding more examples or changing the target language.

Default behaviour: Echo: What is the capital of France?
Polite persona: I would be pleased to assist you with: What is the capital of France?
Few-shot translation: hola mundo


## Exercises

1. **Rewrite a prompt for clarity:** The prompt *"Translate this into another language."* is vague. Rewrite it to be specific about which language and how the translation should be formatted.
2. **Design a zero-shot classification prompt:** Write a prompt that asks a model to classify whether a news article is about sports, politics, technology, or entertainment. Make sure to include instructions and a clear output format.
3. **Create a few-shot prompt:** Using the `few_shot_sentiment` function as a stand-in for a model, design a prompt with two or three examples that help classify the sentiment of a new sentence. Experiment by adding more examples and observing how the classification changes.
4. **Specify an output format:** Write a prompt that instructs the model to extract the names of all countries mentioned in a paragraph and return them as a JSON list.
5. **Positive vs negative instructions:** Write a prompt that tells a model to recommend five books in the science fiction genre and politely decline to discuss personal information. Use positive phrasing to instruct the model what to do.
6. **Iterate and refine:** Start with a basic prompt asking a model to summarise an article. Then iteratively refine your prompt to specify length, target audience, and desired tone. Reflect on how each change could influence the response.

Foundational LLMs & Transformers
1. Vaswani, A., et al. (2017). Attention is All You Need. Advances in Neural Information Processing Systems (NIPS 2017).
2. Brown, T. B., et al. (2020). Language Models are Few-Shot Learners. NeurIPS 2020.
3. Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. NAACL-HLT 2019.
4. OpenAI (2023). GPT-4 Technical Report. arXiv:2303.08774.

5. Touvron, H., et al. (2023). LLaMA 2: Open Foundation and Fine-Tuned Chat Models. Meta AI.


Generative AI & Sampling

6. Goodfellow, I., et al. (2014). Generative Adversarial Nets. NeurIPS 2014.
7. Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer.
8. Neal, R. M. (1993). Probabilistic Inference Using Markov Chain Monte Carlo Methods. Technical Report CRG-TR-93-1, University of Toronto.

Retrieval-Augmented Generation (RAG) & Knowledge Grounding

9. Lewis, P., et al. (2020). Retrieval-Augmented Generation for Knowledge-Intensive NLP. NeurIPS 2020.
10. deepset ai (2023). Haystack: Open-Source Framework for Search and RAG Applications. https://haystack.deepset.ai
11. LangChain (2023). LangChain Documentation and Cookbook. https://python.langchain.com

Evaluation & Safety

12. Papineni, K., et al. (2002). BLEU: A Method for Automatic Evaluation of Machine Translation. ACL 2002.
13. Lin, C.-Y. (2004). ROUGE: A Package for Automatic Evaluation of Summaries. ACL Workshop 2004.
14. OpenAI (2024). Evaluating Model Outputs: Faithfulness and Grounding. OpenAI Docs.
15. Guardrails AI (2024). Open-Source Guardrails Framework. https://github.com/shreyar/guardrails

Prompt Engineering & Instruction Tuning

16. White, J. (2023). The Prompting Guide. https://www.promptingguide.ai
17. Ouyang, L., et al. (2022). Training Language Models to Follow Instructions with Human Feedback. NeurIPS 2022.

Agents & Tool Use

18. Yao, S., et al. (2022). ReAct: Synergizing Reasoning and Acting in Language Models. arXiv:2210.03629.
19. LangChain (2024). LangChain Agents and Tools Documentation.
20. Microsoft (2023). Semantic Kernel Developer Guide. https://learn.microsoft.com/en-us/semantic-kernel/
21. Google DeepMind (2024). Gemini Technical Report. arXiv:2312.11805.

State, Memory & Orchestration

22. LangGraph (2024). Stateful Agent Orchestration Framework. https://langchain-langgraph.vercel.app
23. Park, J. S., et al. (2023). Generative Agents: Interactive Simulacra of Human Behavior. arXiv:2304.03442.

Pedagogical and Course Design References

24. fast.ai (2023). fast.ai Deep Learning Course Notebooks. https://course.fast.ai
25. Ng, A. (2023). DeepLearning.AI Short Courses on Generative AI.
26. MIT 6.S191, Stanford CS324, UC Berkeley CS294-158. (2022–2024). Course Materials and Public Notebooks for ML and LLMs.